# <font color="purple">Adversarial Training of Residual Network</font>

Definition of WideResNet and Architecture:

Residual Networks vs Wide Residual Networks

##  <center>Best Wide Residual Network Experiment </center>


***Road Map***
* Data Preprocessing
* Model Cross Validation Results
* Evaluate the GridSearchCV Results
* Model Training and Learning Curves
* Model Adversarial Training Approach.


**Libraries**

In [ ]:
!pip install -qq -e git+http://github.com/tensorflow/cleverhans.git#egg=cleverhans
import sys
sys.path.append('/content/src/cleverhans')
import cleverhans

In [ ]:
import tensorflow as tf
from cleverhans.future.tf2.attacks import fast_gradient_method
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import Callback, LearningRateScheduler, EarlyStopping
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from wresnet import WideResidualNetwork
from sklearn.model_selection import  KFold
import gzip
import pickle
import numpy as np
from adversarial_training import AdversarialTraining
from training import train
import warnings
warnings.filterwarnings("ignore")

print("\nTensorflow Version: " + tf.__version__)


# defined utility functions
from preprocessing import preprocessing_data

from _utility import lrate, get_adversarial_examples, print_test


Tensorflow Version: 2.3.0


## <font color="green"> Data Preprocessing </font>


* Read Data from File

In [ ]:
def read_data():
    with open("data.pz", 'rb') as file_:
        with gzip.GzipFile(fileobj=file_) as gzf:
            data = pickle.load(gzf, encoding='latin1', fix_imports=True)
    return data
data = read_data()

* Call data preprocessing function

In [ ]:
X, y = preprocessing_data(data)
X_train, X_test, Y_train, y_test = train_test_split(X, y, test_size = 0.1)
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size = 0.1)

## <font color="green"> Utilize Functions </font>

* Flipping the image using data augmentation technique

In [ ]:
generator = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=10,
                               width_shift_range=5./32,
                               height_shift_range=5./32,)

### <font color = "green">Some Parameters Regarding Adversarial Examples</font>

In [ ]:
# predefined epsilon values
epsilon_list = [0.003,0.005,0.01,0.02]

## <font color="purple"> 1.) Model Parameters</font>

Model Parameters of Simple Wide Residual Network According to GridSearch CV

In [ ]:
EPOCHS = 150
BS = 64
init = (32, 32,1)
sgd = SGD(lr=0.01, momentum=0.9)
parameter = {'epochs': EPOCHS, 'batch_size': BS, 'optimizer': sgd}

In [ ]:
import math
def step_decay(epoch):
    """[summary]

    Args:
        epoch (int): epoch number

    Returns:
        float: new learning rate
    """
    initial_lrate = 0.1
    factor = 0.1
    if epoch < 10:
      lrate = initial_lrate
    elif epoch < 20:
      lrate = initial_lrate*math.pow(factor, 1)
    elif epoch < 30:
      lrate = initial_lrate*math.pow(factor, 2)
    elif epoch < 40 :
      lrate = initial_lrate*math.pow(factor, 3)
    else:
      lrate = initial_lrate*math.pow(factor, 4)
    return lrate

lrate = LearningRateScheduler(step_decay)

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
callbacks_list = [lrate, es]
wresnet_ins = WideResidualNetwork(init, 0.0001, 0.9, nb_classes=4, N=2, k=2, dropout=0.0)

In [ ]:
premodel = wresnet_ins.create_wide_residual_network()
premodel.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["acc"])
hst =premodel.fit(generator.flow(x_train, y_train, batch_size=BS),steps_per_epoch=len(x_train) // BS, 
              epochs= EPOCHS, callbacks = callbacks_list,
              validation_data=(x_val, y_val),
              validation_steps=x_val.shape[0] // BS,
              )

In [ ]:
result_df = train(wresnet_ins, premodel, X_train, Y_train, X_test, y_test, EPOCHS, BS, sgd, generator, callbacks_list, epsilon_list)

In [ ]:
result_df["clean_mean"] = np.sum(result_df['acc_clean'])/10.0
result_df["0.003_mean"] = np.sum(result_df['0.003_acc'])/10.0
result_df["0.005_mean"] = np.sum(result_df['0.005_acc'])/10.0
result_df["0.02_mean"] = np.sum(result_df['0.02_acc'])/10.0
result_df["0.01_mean"] = np.sum(result_df['0.01_acc'])/10.0

In [ ]:
column = ["clean_mean","0.003_mean","0.005_mean","0.02_mean","0.01_mean"]
result_df[column].head(1)

,clean_mean,0.003_mean,0.005_mean,0.02_mean,0.01_mean
0,0.738743,0.680628,0.65096,0.577661,0.434555


## <font color="purple">2.) Adversarial Training on Baseline Model</font>

In [ ]:
adversarial_training =  AdversarialTraining(parameter)
result_adv_df = adversarial_training.train(wresnet_ins,premodel, X_train, Y_train, X_test, y_test, epsilon_list, callbacks_list)

### <font color="purple"> Results of Adversarial Training</font>

In [ ]:
result_adv_df["clean_mean"] = np.sum(result_adv_df['acc_clean'])/10.0
result_adv_df["0.003_mean"] = np.sum(result_adv_df['0.003_acc'])/10.0
result_adv_df["0.005_mean"] = np.sum(result_adv_df['0.005_acc'])/10.0
result_adv_df["0.02_mean"] = np.sum(result_adv_df['0.02_acc'])/10.0
result_adv_df["0.01_mean"] = np.sum(result_adv_df['0.01_acc'])/10.0

In [ ]:
result_adv_df[column].head(1)

,clean_mean,0.003_mean,0.005_mean,0.02_mean,0.01_mean
0,0.730366,0.727051,0.723735,0.703316,0.663525


# <font color="blue">Compare Non-Adversarial Training with Adversarial Training</font>

In [ ]:
result_adv_df[column].head(1)

,clean_mean,0.003_mean,0.005_mean,0.02_mean,0.01_mean
0,0.730366,0.727051,0.723735,0.703316,0.663525


In [ ]:
result_df[column].head(1)

,clean_mean,0.003_mean,0.005_mean,0.02_mean,0.01_mean
0,0.738743,0.680628,0.65096,0.577661,0.434555


In [ ]:
result_df.to_json('wrn_2_n_adv.json.gz', orient='records', lines=True, compression='gzip')

In [ ]:
result_adv_df.to_json('wrn_2_adv.json.gz', orient='records', lines=True, compression='gzip')

# <font color="blue">Conclusion</font>